In [1]:
import sys
import pandas as pd
sys.path.append('/home/wangs/rs/lib')
# sys.path.append('/home/wangs/rs/qza')
import os
import numpy as np
from multiprocessing import Pool
from tqdm import tqdm
import zipfile
import datetime
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import imp
import ff

In [2]:
# 所有加密货币的名字
currency_names=['BTC','ETH','XRP','BNB','TRX','ADA','ICP','APE','ARB','ARPA','ATOM','AVAX','BAKE','DOT','ETH',"FIL",'INJ','LINK','MKR','NEAR','NEO','RUNE','SEI','SOL','SUI','UNI','VET','WLD']
# currency_names = ['BTC', 'ETH', 'XRP', 'LTC', 'BCH', 'EOS', 'BNB', 'XLM', 'TRX', 'ADA']
# 读取一个币种的高开低收成交量
coin = "BTC"
ff.read_binance(coin)
# 预处理要用到的工具函数
freq = 15 # 15分钟为周期采样
resample_rule = "{}min".format(freq)

# Convert index from str into pd.Datetime
def timing(df):
    df.index = pd.to_datetime(df.index)
    df.index.name = None
    return df

def naming(df, name):
    df.name = name
    return df

def phase_transformer(s): # s = "20220101 010300"
    minute = int(s[11:13])
    return f"{s[:11]}{minute-minute%freq:02d}"

start, end = "20200101", "20240101"
# 读取数据
def currency_data(currency_name):
    global start, end
    data = ff.read_binance(currency_name).rename(columns={'o':'open', 'h':'high', 'l':'low', 'c':'close', 'v':'volume'}).loc[start:end]
    data['tradedate'] = data.index.str[:8]
    data['tradehour'] = data.index.str[:11]
    data['tradephase'] = data.index.str[:15]
    data['tradephase'] = data.tradephase.apply(phase_transformer) # 如果要重采样的话，使用函数phase_transformer进行时间切分
    data = timing(data) # 将data的行索引转化为pd.datetime形式
    return data.reindex(pd.date_range(start=pd.to_datetime(start), 
                                      end=pd.to_datetime(end), 
                                      freq="1min", inclusive="left")) # 补充上缺失的时间索引

In [3]:
data = [currency_data(currency_name)['close'] for currency_name in currency_names]
data = pd.DataFrame(data).T
data.columns = currency_names

In [49]:
import psutil
import os
print(u'当前进程的内存使用：%.4f GB' % (psutil.Process(os.getpid()).memory_info().rss / 1024 ** 3) )
info = psutil.virtual_memory()
print(u'电脑总内存：%.4f GB' % (info.total / 1024 ** 3))
print(u'当前使用的总内存占比：', info.percent)
print(u'cpu个数：', psutil.cpu_count())

当前进程的内存使用：10.9527 GB
电脑总内存：1007.4503 GB
当前使用的总内存占比： 67.7
cpu个数： 256


In [11]:
currency_data('BTC').iloc[:,0:4].stack().rolling(window = 20).mean().unstack()['close']

2020-01-01 00:00:00           NaN
2020-01-01 00:01:00           NaN
2020-01-01 00:02:00           NaN
2020-01-01 00:03:00           NaN
2020-01-01 00:04:00     7210.5970
                          ...    
2023-12-31 23:55:00    42461.3105
2023-12-31 23:56:00    42453.6780
2023-12-31 23:57:00    42451.0675
2023-12-31 23:58:00    42452.0050
2023-12-31 23:59:00    42457.1585
Name: close, Length: 2101493, dtype: float64

In [19]:
def stock_mean(n):
    try:
        stock_mean = currency_data(n).iloc[:,0:4].stack().rolling(window = 20).mean().unstack()['close']
        # stock_mean = data.stack().rolling(window = 20).mean().unstack()['high']
    except:
        return pd.Series(np.nan,index = data.index).values
    return pd.Series(stock_mean,index = data.index).values

In [21]:
with Pool(24) as p:
    res_lst = list(tqdm(p.imap(stock_mean,currency_names),total = len(currency_names)))

100%|████████████████████████████████████████████████████████████████| 28/28 [00:10<00:00,  2.74it/s]


In [22]:
data_mean = np.vstack(res_lst)

In [24]:
def stock_std(n):
    try:
        stock_std = currency_data(n).iloc[:,0:4].stack().rolling(window = 20).std().unstack()['close']
        # stock_std = data.stack().rolling(window = 20).std().unstack()['high']
    except:
        return pd.Series(np.nan,index = data.index).values
    return pd.Series(stock_std,index = data.index).values
with Pool(24) as p:
    res_lst = list(tqdm(p.imap(stock_std,currency_names),total = len(currency_names)))

100%|████████████████████████████████████████████████████████████████| 28/28 [00:10<00:00,  2.58it/s]


In [75]:
data_mean[0][:100]

array([      nan,       nan,       nan,       nan, 7210.597 , 7214.561 ,
       7215.8725, 7215.3255, 7213.6695, 7213.032 , 7212.396 , 7210.8045,
       7210.1945, 7210.443 , 7209.664 , 7208.957 , 7209.221 , 7208.3855,
       7207.799 , 7209.2135, 7211.643 , 7213.6165, 7216.1085, 7218.4445,
       7219.42  , 7218.73  , 7218.561 , 7218.42  , 7218.1645, 7218.3285,
       7219.4705, 7220.709 , 7221.3725, 7221.657 , 7221.702 , 7220.963 ,
       7218.8955, 7217.503 , 7216.984 , 7215.609 , 7214.6865, 7214.855 ,
       7215.597 , 7215.7145, 7216.322 , 7216.253 , 7216.164 , 7215.668 ,
       7215.6665, 7215.4625, 7215.45  , 7215.6295, 7215.8235, 7215.6535,
       7215.901 , 7216.427 , 7216.25  , 7215.3215, 7214.534 , 7213.737 ,
       7212.5345, 7211.1555, 7210.1745, 7209.0135, 7207.5295, 7206.2495,
       7205.978 , 7206.6205, 7207.343 , 7208.0895, 7209.054 , 7209.4685,
       7209.2625, 7209.06  , 7208.755 , 7207.903 , 7207.1245, 7206.7215,
       7206.478 , 7206.5715, 7207.6705, 7208.605 , 

In [25]:
data_std = np.vstack(res_lst)

In [28]:
up_status = data.values.T > (data_std + data_mean)
down_status = data.values.T < (data_mean - data_std)
mid_status = (data.values.T <= (data_std + data_mean)) * (data.values.T >= (data_mean - data_std))

In [32]:
mid_status

array([[False, False, False, ...,  True,  True,  True],
       [False, False, False, ..., False, False,  True],
       [False, False, False, ...,  True,  True,  True],
       ...,
       [False, False, False, ..., False,  True,  True],
       [False, False, False, ...,  True,  True,  True],
       [False, False, False, ..., False,  True,  True]])

In [51]:
def volume_min(n):
    try:
        test = currency_data(n).iloc[:,4]
        volume_daily = test.rolling(window = 1440).sum()
        volume_daily = volume_daily[volume_daily.index.strftime("%Y-%m-%d %H:%M:%S").str.endswith('23:59:00')]
        result = test / np.repeat(volume_daily.values,1440)
        return pd.Series(result,index = data.index).values
    except:
        return pd.Series(np.nan,index = data_all.index).values

In [52]:
with Pool(24) as p:
    res_lst = list(tqdm(p.imap(volume_min,currency_names),total = len(currency_names)))

100%|████████████████████████████████████████████████████████████████| 28/28 [00:13<00:00,  2.13it/s]


In [53]:
data_volume_min = np.vstack(res_lst)

In [54]:
Cor_up_stock_volume = data_volume_min * up_status
Cor_down_stock_volume = data_volume_min * down_status
Cor_mid_stock_volume = data_volume_min * mid_status

In [59]:
def stock_corr(n):
    corr = []
    for i in range(1461):
        corr.append(np.corrcoef(single_stock_volume[n][i],relate_stock_volume[n][i])[0,1])
    return corr

In [61]:
single_stock_volume = np.reshape(data_volume_min,(28,-1,1440))

In [62]:
relate_stock_volume = np.reshape((up_status * np.nansum(Cor_up_stock_volume,axis = 0) + mid_status * np.nansum(Cor_mid_stock_volume,axis = 0) + down_status* np.nansum(Cor_down_stock_volume,axis = 0)),(28,-1,1440))

In [63]:
with Pool(24) as p:
    res_lst = list(tqdm(p.imap(stock_corr,range(28)),total = len(range(28))))

100%|███████████████████████████████████████████████████████████████| 28/28 [00:00<00:00, 121.38it/s]


In [64]:
corr_result = np.vstack(res_lst)

In [67]:
np.shape(corr_result)

(28, 1461)

In [76]:
factor = pd.DataFrame(ff.rolling_window(corr_result,window = 20).mean(axis = -1) + ff.rolling_window(corr_result,window = 20).std(axis = -1))/2